## Text Classification

In [1]:
!pip install -q tensorflow-hub
!pip install -q tfds-nightly

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np

### Download and Explore Dataset

In [3]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews',
                                                   split=('train[:60%]', 'train[60%:]', 'test'),
                                                   as_supervised=True)

Shuffling and writing examples to /Users/kumarujjawal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIZUS9Y/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/kumarujjawal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIZUS9Y/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/kumarujjawal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIZUS9Y/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/kumarujjawal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### Build the Model

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
history = model.fit(train_data.shuffle(10000).batch(512), 
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 5s 174ms/step - loss: 0.8099 - accuracy: 0.5408 - val_loss: 0.6788 - val_accuracy: 0.5889
Epoch 2/20
30/30 [==============================] - 4s 139ms/step - loss: 0.6517 - accuracy: 0.5837 - val_loss: 0.6308 - val_accuracy: 0.5980
Epoch 3/20
30/30 [==============================] - 4s 144ms/step - loss: 0.6191 - accuracy: 0.6206 - val_loss: 0.6070 - val_accuracy: 0.6307
Epoch 4/20
30/30 [==============================] - 4s 139ms/step - loss: 0.5940 - accuracy: 0.6527 - val_loss: 0.5863 - val_accuracy: 0.6603
Epoch 5/20
30/30 [==============================] - 4s 124ms/step - loss: 0.5715 - accuracy: 0.6790 - val_loss: 0.5666 - val_accuracy: 0.6764
Epoch 6/20
30/30 [==============================] - 4s 126ms/step - loss: 0.5485 - accuracy: 0.7021 - val_loss: 0.5470 - val_accuracy: 0.7009
Epoch 7/20
30/30 [==============================] - 4s 123ms/step - loss: 0.5259 - accuracy: 0.7219 - val_loss: 0.5268 - val_accuracy: 0.7228
Epoch 

### Evaluate the Model

In [12]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name,value in zip(model.metrics_names, results):
    print("%s: %.3f" %(name, value))

loss: 0.327
accuracy: 0.849


## End